<AsyncResult(_push): pending>

In [1]:
import pandas as pd





News_videos = "News_person_search_results.csv"
Entertainment_videos = "Other_Entertainment_person_search_results.csv"
UK_Show_videos= "UK_Shows_person_search_results.csv"
US_Show_videos="US_Shows_person_search_results.csv"

#list of urls

News_urls_and_persons = []
Entertainment_urls_and_persons = []
UK_urls_and_persons = []
US_urls_and_persons = []

News_urls = []
Entertainment_urls = []
UK_urls = []
US_urls = []
def SaveFunction(file, url_list, urls_and_persons):
    df = pd.read_csv(file, dtype = {
        'video_id': str,
        'url': str,
        'title': str,
        'channel_name': str,
        'published_at': str,
        'view_count': 'Int64',
        'persons_found': str,
        'total_persons': 'Int64'
    })
    df['persons_found_list'] = (df['persons_found'].fillna('')
                     .apply(lambda x: [name.strip() for name in x.split(',') if name.strip()])
                    )
    for url, persons in zip(df['url'], df['persons_found_list']):
        url_list.append(url)
        record = {
            "url": url,
            "persons_found": persons
        }
        urls_and_persons.append(record)

SaveFunction(News_videos, News_urls, News_urls_and_persons)
SaveFunction(Entertainment_videos, Entertainment_urls, Entertainment_urls_and_persons)
SaveFunction(UK_Show_videos, UK_urls, UK_urls_and_persons)
SaveFunction(US_Show_videos, US_urls, US_urls_and_persons)

print(f"We have {len(News_urls)} News videos to process.")
print(f"We have {len(Entertainment_urls)} Entertainment videos to process.")
print(f"We have {len(UK_urls)} UK Show videos to process.")
print(f"We have {len(US_urls)} US Show videos to process.")
print(f"In total, there are {len(News_urls)+len(Entertainment_urls)+len(UK_urls)+ len(US_urls)} videos.")
print(f"Sample persons_found list: {UK_urls_and_persons[:3]}")

We have 22716 News videos to process.
We have 404 Entertainment videos to process.
We have 518 UK Show videos to process.
We have 3200 US Show videos to process.
In total, there are 26838 videos.
Sample persons_found list: [{'url': 'https://youtu.be/LOA-rEMU8pA', 'persons_found': ['Ben Affleck', 'Margot Robbie', 'Ryan Gosling']}, {'url': 'https://youtu.be/t9H3_ewSKI8', 'persons_found': ['Chris Hemsworth', 'Hugh Jackman', 'Tom Cruise']}, {'url': 'https://youtu.be/rFnB7gTk8L8', 'persons_found': ['Nicole Kidman', 'Ryan Gosling']}]


In [2]:
import os
import time
import random
import logging
import json
from xml.parsers.expat import ExpatError
from xml.etree.ElementTree import ParseError
from langchain_community.document_loaders import YoutubeLoader
from langchain.schema import Document   # for reconstructing

RETRIES    = 3
BASE_DELAY = 2.0
STATE_FILE_1 = "News_yt_state.json"
logging.basicConfig(
    filename=None,
    level=logging.INFO,
    format="%(asctime)s %(levelname)-8s %(message)s",
    datefmt = "%Y-%m-%d %H:%M:%S"
)

# ─── LOAD STATE ───────────────────────────────────────────────────────────
if os.path.exists(STATE_FILE_1):
    with open(STATE_FILE_1, "r", encoding="utf-8") as f:
        state = json.load(f)
    all_transcripts = [
        Document(page_content=d["page_content"], metadata=d["metadata"])
        for d in state.get("all_transcripts", [])
    ]
    not_working    = state.get("not_working", [])
    processed_urls = set(state.get("processed_urls", []))
    logging.info(f"Resumed state: {len(all_transcripts)} docs, "
                 f"{len(not_working)} failures, {len(processed_urls)} done")
else:
    all_transcripts, not_working, processed_urls = [], [], set()

def save_state():
    payload = {
        "all_transcripts": [
            {"page_content": doc.page_content, "metadata": doc.metadata}
            for doc in all_transcripts
        ],
        "not_working": list(not_working),
        "processed_urls": list(processed_urls),
    }
    with open(STATE_FILE_1, "w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, indent=2)

def load_with_retry(url):
    for attempt in range(1, RETRIES + 1):
        try:
            docs = YoutubeLoader.from_youtube_url(
                url, add_video_info=True, translation="en"
            ).load()
            logging.info(f"✅ [{url}] loaded on attempt {attempt}")
            return docs
        except (ExpatError, ParseError) as e:
            logging.warning(f"⚠️  [{url}] parse error on attempt {attempt}: {e}")
        except Exception as e:
            logging.error(f"❌ [{url}] unexpected error: {e}", exc_info=True)
            break
        backoff = BASE_DELAY * (2 ** (attempt - 1)) + random.random()
        time.sleep(backoff)
    logging.error(f"✖️  [{url}] failed after {RETRIES} attempts")
    return []

# ─── MAIN LOOP ────────────────────────────────────────────────────────────
for url in News_urls:    
    if url in processed_urls:
        continue
        
    docs = load_with_retry(url)
    if docs:
        all_transcripts.extend(docs)
    else:
        not_working.append(url)
    processed_urls.add(url)
    save_state()        # ← write JSON after each URL
    time.sleep(1.5)

logging.info(f"Done: {len(all_transcripts)} docs, {len(not_working)} failures")

2025-05-29 15:10:56 INFO     Resumed state: 1614 docs, 134 failures, 1748 done
2025-05-29 15:10:59 INFO     ✅ [https://youtu.be/N-9z9_OX5vw] loaded on attempt 1
2025-05-29 15:11:05 INFO     ✅ [https://youtu.be/JnvnBDUZX4g] loaded on attempt 1
2025-05-29 15:11:10 ERROR    ❌ [https://youtu.be/VvKDjNkP0x8] unexpected error: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=VvKDjNkP0x8! This is most likely caused by:

YouTube is blocking requests from your IP. This usually is due to one of the following reasons:
- You have done too many requests and your IP has been blocked by YouTube
- You are doing requests from an IP belonging to a cloud provider (like AWS, Google Cloud Platform, Azure, etc.). Unfortunately, most IPs from cloud providers are blocked by YouTube.

There are two things you can do to work around this:
1. Use proxies to hide your IP address, as explained in the "Working around IP bans" section of the README (https://github.com/jdepoix/youtube-tra

2025-05-29 15:11:36 ERROR    ✖️  [https://youtu.be/ROBO6_v40MU] failed after 3 attempts
2025-05-29 15:11:40 INFO     ✅ [https://youtu.be/NR-uhyZ0W2c] loaded on attempt 1
2025-05-29 15:11:45 ERROR    ❌ [https://youtu.be/ZLcyBj-ZuIM] unexpected error: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=ZLcyBj-ZuIM! This is most likely caused by:

YouTube is blocking requests from your IP. This usually is due to one of the following reasons:
- You have done too many requests and your IP has been blocked by YouTube
- You are doing requests from an IP belonging to a cloud provider (like AWS, Google Cloud Platform, Azure, etc.). Unfortunately, most IPs from cloud providers are blocked by YouTube.

There are two things you can do to work around this:
1. Use proxies to hide your IP address, as explained in the "Working around IP bans" section of the README (https://github.com/jdepoix/youtube-transcript-api?tab=readme-ov-file#working-around-ip-bans-requestblocked-or-i

2025-05-29 15:11:50 ERROR    ✖️  [https://youtu.be/lzK3x3_5HVo] failed after 3 attempts
2025-05-29 15:11:55 INFO     ✅ [https://youtu.be/Nd2XvMTPWh4] loaded on attempt 1
2025-05-29 15:12:00 ERROR    ❌ [https://youtu.be/mIyJttK8aq4] unexpected error: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=mIyJttK8aq4! This is most likely caused by:

YouTube is blocking requests from your IP. This usually is due to one of the following reasons:
- You have done too many requests and your IP has been blocked by YouTube
- You are doing requests from an IP belonging to a cloud provider (like AWS, Google Cloud Platform, Azure, etc.). Unfortunately, most IPs from cloud providers are blocked by YouTube.

There are two things you can do to work around this:
1. Use proxies to hide your IP address, as explained in the "Working around IP bans" section of the README (https://github.com/jdepoix/youtube-transcript-api?tab=readme-ov-file#working-around-ip-bans-requestblocked-or-i

2025-05-29 15:12:05 ERROR    ✖️  [https://youtu.be/_21pbizBR9Y] failed after 3 attempts
2025-05-29 15:12:10 INFO     ✅ [https://youtu.be/Jr7y6tZ3IL4] loaded on attempt 1
2025-05-29 15:12:16 ERROR    ❌ [https://youtu.be/-2DKPhdxv0s] unexpected error: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=-2DKPhdxv0s! This is most likely caused by:

YouTube is blocking requests from your IP. This usually is due to one of the following reasons:
- You have done too many requests and your IP has been blocked by YouTube
- You are doing requests from an IP belonging to a cloud provider (like AWS, Google Cloud Platform, Azure, etc.). Unfortunately, most IPs from cloud providers are blocked by YouTube.

There are two things you can do to work around this:
1. Use proxies to hide your IP address, as explained in the "Working around IP bans" section of the README (https://github.com/jdepoix/youtube-transcript-api?tab=readme-ov-file#working-around-ip-bans-requestblocked-or-i

2025-05-29 15:12:21 ERROR    ✖️  [https://youtu.be/iYGHrt1HjI8] failed after 3 attempts
2025-05-29 15:12:27 INFO     ✅ [https://youtu.be/NC5NZPrxbHk] loaded on attempt 1
2025-05-29 15:12:32 WARNING  ⚠️  [https://youtu.be/pcuwZ8zk2ng] parse error on attempt 1: no element found: line 1, column 0
2025-05-29 15:12:38 INFO     ✅ [https://youtu.be/pcuwZ8zk2ng] loaded on attempt 2
2025-05-29 15:12:43 ERROR    ❌ [https://youtu.be/DcvdOWitb3I] unexpected error: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=DcvdOWitb3I! This is most likely caused by:

YouTube is blocking requests from your IP. This usually is due to one of the following reasons:
- You have done too many requests and your IP has been blocked by YouTube
- You are doing requests from an IP belonging to a cloud provider (like AWS, Google Cloud Platform, Azure, etc.). Unfortunately, most IPs from cloud providers are blocked by YouTube.

There are two things you can do to work around this:
1. Use proxi

KeyboardInterrupt: 

In [ ]:
import os
import time
import random
import logging
import json
from xml.parsers.expat import ExpatError
from xml.etree.ElementTree import ParseError
from langchain_community.document_loaders import YoutubeLoader
from langchain.schema import Document   # for reconstructing


RETRIES    = 3
BASE_DELAY = 2.0
STATE_FILE_2 = "Entartainment_yt_state.json"
logging.basicConfig(
    filename=None,
    level=logging.INFO,
    format="%(asctime)s %(levelname)-8s %(message)s",
    datefmt = "%Y-%m-%d %H:%M:%S"
)

# ─── LOAD STATE ───────────────────────────────────────────────────────────
if os.path.exists(STATE_FILE_2):
    with open(STATE_FILE_2, "r", encoding="utf-8") as f:
        state = json.load(f)
    all_transcripts = [
        Document(page_content=d["page_content"], metadata=d["metadata"])
        for d in state.get("all_transcripts", [])
    ]
    not_working    = state.get("not_working", [])
    processed_urls = set(state.get("processed_urls", []))
    logging.info(f"Resumed state: {len(all_transcripts)} docs, "
                 f"{len(not_working)} failures, {len(processed_urls)} done")
else:
    all_transcripts, not_working, processed_urls = [], [], set()

def save_state():
    payload = {
        "all_transcripts": [
            {"page_content": doc.page_content, "metadata": doc.metadata}
            for doc in all_transcripts
        ],
        "not_working": list(not_working),
        "processed_urls": list(processed_urls),
    }
    with open(STATE_FILE_2, "w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, indent=2)

def load_with_retry(url):
    for attempt in range(1, RETRIES + 1):
        try:
            docs = YoutubeLoader.from_youtube_url(
                url, add_video_info=True, translation="en"
            ).load()
            logging.info(f"✅ [{url}] loaded on attempt {attempt}")
            return docs
        except (ExpatError, ParseError) as e:
            logging.warning(f"⚠️  [{url}] parse error on attempt {attempt}: {e}")
        except Exception as e:
            logging.error(f"❌ [{url}] unexpected error: {e}", exc_info=True)
            break
        backoff = BASE_DELAY * (2 ** (attempt - 1)) + random.random()
        time.sleep(backoff)
    logging.error(f"✖️  [{url}] failed after {RETRIES} attempts")
    return []

# ─── MAIN LOOP ────────────────────────────────────────────────────────────
for url in Entertainment_urls:    
    if url in processed_urls:
        continue
        
    docs = load_with_retry(url)
    if docs:
        all_transcripts.extend(docs)
    else:
        not_working.append(url)
    processed_urls.add(url)
    save_state()        # ← write JSON after each URL
    time.sleep(1.5)

logging.info(f"Done: {len(all_transcripts)} docs, {len(not_working)} failures")


In [ ]:
%%px --targets 2

import os
import time
import random
import logging
import json
from xml.parsers.expat import ExpatError
from xml.etree.ElementTree import ParseError
from langchain_community.document_loaders import YoutubeLoader
from langchain.schema import Document   # for reconstructing


RETRIES    = 3
BASE_DELAY = 2.0
STATE_FILE_3 = "UK_Shows_yt_state.json"
logging.basicConfig(
    filename=None,
    level=logging.INFO,
    format="%(asctime)s %(levelname)-8s %(message)s",
    datefmt = "%Y-%m-%d %H:%M:%S"
)

# ─── LOAD STATE ───────────────────────────────────────────────────────────
if os.path.exists(STATE_FILE_3):
    with open(STATE_FILE_3, "r", encoding="utf-8") as f:
        state = json.load(f)
    all_transcripts = [
        Document(page_content=d["page_content"], metadata=d["metadata"])
        for d in state.get("all_transcripts", [])
    ]
    not_working    = state.get("not_working", [])
    processed_urls = set(state.get("processed_urls", []))
    logging.info(f"Resumed state: {len(all_transcripts)} docs, "
                 f"{len(not_working)} failures, {len(processed_urls)} done")
else:
    all_transcripts, not_working, processed_urls = [], [], set()

def save_state():
    payload = {
        "all_transcripts": [
            {"page_content": doc.page_content, "metadata": doc.metadata}
            for doc in all_transcripts
        ],
        "not_working": list(not_working),
        "processed_urls": list(processed_urls),
    }
    with open(STATE_FILE_3, "w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, indent=2)

def load_with_retry(url):
    for attempt in range(1, RETRIES + 1):
        try:
            docs = YoutubeLoader.from_youtube_url(
                url, add_video_info=True, translation="en"
            ).load()
            logging.info(f"✅ [{url}] loaded on attempt {attempt}")
            return docs
        except (ExpatError, ParseError) as e:
            logging.warning(f"⚠️  [{url}] parse error on attempt {attempt}: {e}")
        except Exception as e:
            logging.error(f"❌ [{url}] unexpected error: {e}", exc_info=True)
            break
        backoff = BASE_DELAY * (2 ** (attempt - 1)) + random.random()
        time.sleep(backoff)
    logging.error(f"✖️  [{url}] failed after {RETRIES} attempts")
    return []

# ─── MAIN LOOP ────────────────────────────────────────────────────────────
for url in UK_urls:    
    if url in processed_urls:
        continue
        
    docs = load_with_retry(url)
    if docs:
        all_transcripts.extend(docs)
    else:
        not_working.append(url)
    processed_urls.add(url)
    save_state()        # ← write JSON after each URL
    time.sleep(1.5)

logging.info(f"Done: {len(all_transcripts)} docs, {len(not_working)} failures")

In [ ]:
%%px --targets 3

import os
import time
time.sleep(4)
import random
import logging
import json
from xml.parsers.expat import ExpatError
from xml.etree.ElementTree import ParseError
from langchain_community.document_loaders import YoutubeLoader
from langchain.schema import Document   # for reconstructing

RETRIES    = 3
BASE_DELAY = 2.0
STATE_FILE_4 = "US_Shows_yt_state.json"
logging.basicConfig(
    filename=None,
    level=logging.INFO,
    format="%(asctime)s %(levelname)-8s %(message)s",
    datefmt = "%Y-%m-%d %H:%M:%S"
)

# ─── LOAD STATE ───────────────────────────────────────────────────────────
if os.path.exists(STATE_FILE_4):
    with open(STATE_FILE_4, "r", encoding="utf-8") as f:
        state = json.load(f)
    all_transcripts = [
        Document(page_content=d["page_content"], metadata=d["metadata"])
        for d in state.get("all_transcripts", [])
    ]
    not_working    = state.get("not_working", [])
    processed_urls = set(state.get("processed_urls", []))
    logging.info(f"Resumed state: {len(all_transcripts)} docs, "
                 f"{len(not_working)} failures, {len(processed_urls)} done")
else:
    all_transcripts, not_working, processed_urls = [], [], set()

def save_state():
    payload = {
        "all_transcripts": [
            {"page_content": doc.page_content, "metadata": doc.metadata}
            for doc in all_transcripts
        ],
        "not_working": list(not_working),
        "processed_urls": list(processed_urls),
    }
    with open(STATE_FILE_4, "w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, indent=2)

def load_with_retry(url):
    for attempt in range(1, RETRIES + 1):
        try:
            docs = YoutubeLoader.from_youtube_url(
                url, add_video_info=True, translation="en"
            ).load()
            logging.info(f"✅ [{url}] loaded on attempt {attempt}")
            return docs
        except (ExpatError, ParseError) as e:
            logging.warning(f"⚠️  [{url}] parse error on attempt {attempt}: {e}")
        except Exception as e:
            logging.error(f"❌ [{url}] unexpected error: {e}", exc_info=True)
            break
        backoff = BASE_DELAY * (2 ** (attempt - 1)) + random.random()
        time.sleep(backoff)
    logging.error(f"✖️  [{url}] failed after {RETRIES} attempts")
    return []

# ─── MAIN LOOP ────────────────────────────────────────────────────────────
for url in US_urls:    
    if url in processed_urls:
        continue
        
    docs = load_with_retry(url)
    if docs:
        all_transcripts.extend(docs)
    else:
        not_working.append(url)
    processed_urls.add(url)
    save_state()        # ← write JSON after each URL
    time.sleep(1.5)

logging.info(f"Done: {len(all_transcripts)} docs, {len(not_working)} failures")